In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def load_matrix(filepath, ages=range(35,65+1), years=range(2015,2022+1), transpose=False):
    df = pd.read_csv(filepath, index_col=0)
    if transpose:
        df = df.T
    df.columns = [int(col) if isinstance(col, int) or col.isdigit() else int(col[4:]) for col in df.columns]
    df.index = [int(idx) if isinstance(idx, int) or idx.isdigit() else int(idx[5:]) for idx in df.index]
    # Find common years and ages
    common_years = df.index.intersection(years)
    common_ages = df.columns.intersection(ages)
    # Filter dataframes to only include common years and ages
    mat = df.loc[common_years, common_ages].to_numpy()
    return mat, common_ages, common_years

In [ ]:

names = ["IRL_F", "IRL_M", "UK_F", "UK_M"]
ages=range(35, 65+1)
years=range(2015, 2022+1)

for name in names:
    QBrut, QBrut_ages, QBrut_years = load_matrix(f"./matrices/BRUT_LISSES_{name}.csv", ages=ages, years=years)
    QHmd, QHmd_ages, QHmd_years = load_matrix(f"./HMD_inputs/matrices/HMD_{name}.csv", ages=ages, years=years, transpose=True)

    n_years = len(years)
    ncols = 3
    nrows = (n_years + ncols - 1) // ncols  # Compute the number of rows needed
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 6, nrows * 4), sharey=True)
    plt.subplots_adjust(wspace=0.1, hspace=0.4)

    # Flatten the axes array for easy iteration (in case of multiple rows)
    axes = axes.flatten()

    # Plotting
    for year_idx, (ax, year) in enumerate(zip(axes, years)):
        if year in QBrut_years:
            ax.plot(QBrut_ages, QBrut[year_idx], color='blue', label='Taux bruts')
        if year in QHmd_years:
            ax.plot(QHmd_ages, QHmd[year_idx], color='red', label='Taux HMD')
        ax.set_xlabel('Age')
        ax.set_title(f"Année {year}")
        if year_idx == 0:
            ax.set_ylabel('Taux')
            ax.legend()
    # Hide any unused subplots
    for ax in axes[n_years:]:
        ax.axis('off')

    fig.suptitle(f"Comparaison des taux HMD avec les taux bruts {name}")
    # Adjust layout
    plt.savefig(f"images/HDM_BRUT_{name}_comparaison.png", bbox_inches='tight')
    plt.show()